# 02. Regrid & Match

Regrid CMIP6 to the observation grid (xESMF), crop, and align times.

In [ ]:

import xarray as xr
import xesmf as xe
from pathlib import Path
from src.utils import match_to_reference

ds_cmip = xr.open_dataset('data/processed/cmip6_ea_clean.nc')
ds_obs = xr.open_dataset('data/processed/obs_ea_clean.nc')
print(ds_cmip)
print(ds_obs)


In [ ]:

# Build regridder (bilinear by default). First call builds weights file.
regridder = xe.Regridder(ds_cmip, ds_obs, method='bilinear', reuse_weights=False)
pr_rg = regridder(ds_cmip['pr'])
pr_rg.name = 'pr'
ds_cmip_rg = pr_rg.to_dataset()
ds_cmip_rg.to_netcdf('data/processed/cmip6_ea_regridded.nc')
ds_cmip_rg


In [ ]:

# Align time to observation timestamps 
pr_cmip_match = match_to_reference(ds_cmip_rg['pr'], ds_obs['pr'] if 'pr' in ds_obs else list(ds_obs.data_vars.values())[0])
obs_match = ds_obs.sel(time=pr_cmip_match.time)
pr_cmip_match.to_netcdf('data/processed/cmip6_ea_matched.nc')
obs_match.to_netcdf('data/processed/obs_ea_matched.nc')
pr_cmip_match, obs_match
